50 Iteration Experiment

L1 Lasso

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import statistics
import math

misclassification_rate_l1_lasso = []

# Define column names based on the UCI Machine Learning Repository description
column_names = [
    "word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d",
    "word_freq_our", "word_freq_over", "word_freq_remove", "word_freq_internet",
    "word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will",
    "word_freq_people", "word_freq_report", "word_freq_addresses", "word_freq_free",
    "word_freq_business", "word_freq_email", "word_freq_you", "word_freq_credit",
    "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money",
    "word_freq_hp", "word_freq_hpl", "word_freq_george", "word_freq_650",
    "word_freq_lab", "word_freq_labs", "word_freq_telnet", "word_freq_857",
    "word_freq_data", "word_freq_415", "word_freq_85", "word_freq_technology",
    "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct",
    "word_freq_cs", "word_freq_meeting", "word_freq_original", "word_freq_project",
    "word_freq_re", "word_freq_edu", "word_freq_table", "word_freq_conference",
    "char_freq_;", "char_freq_(", "char_freq_[", "char_freq_!", "char_freq_$",
    "char_freq_#", "capital_run_length_average", "capital_run_length_longest",
    "capital_run_length_total", "spam"
]

file_path = "C:/Users/91959/Desktop/CODE/Robust-Logistic-Regression-with-Shift-Parameter-Estimation/Robust Logistic Regression [DATA DIRECTORY]/Linear Case/spambase/spambase.data"

df = pd.read_csv(file_path, header=None, names=column_names)

# Separate features and target variable
X = df.drop('spam', axis=1)  # Features
y = df['spam']  # Target variable

# Standardize the feature set (zero mean, unit variance)
scaler = StandardScaler()
X = scaler.fit_transform(X)

def introduce_label_noise(y, noise_percentage=0.1):
    """
    Introduces label noise by flipping a percentage of majority class labels to the minority class.

    Args:
        y (pd.Series): The target variable.
        noise_percentage (float): The percentage of majority class labels to flip (e.g., 0.1 for 10%).

    Returns:
        pd.Series: The target variable with label noise.
    """

    value_counts = y.value_counts()
    majority_class = value_counts.idxmax()
    minority_class = value_counts.idxmin()

    # print("Original class distribution:")
    # print(value_counts)

    majority_indices = y[y == majority_class].index
    num_noise = int(len(majority_indices) * noise_percentage)

    noise_indices = np.random.choice(majority_indices, num_noise, replace=False)

    y_noisy = y.copy()
    y_noisy.loc[noise_indices] = minority_class

    # print("\nClass distribution after introducing noise:")
    # print(y_noisy.value_counts())

    return y_noisy

# Methods for LogReg
# ===========================
# SIGMOID FUNCTION
# ===========================
def sigmoid(z):
    """Compute the sigmoid function."""
    return 1 / (1 + np.exp(-z))

# ===========================
# THRESHOLDING FUNCTIONS FOR SHIFT PARAMETERS
# ===========================
def soft_threshold(u, lambda_, a):
    """Soft thresholding function for shift parameter estimation."""
    return a * np.minimum(u + lambda_, 0)

def hard_threshold(u, lambda_, a):
    """Hard thresholding function for shift parameter estimation."""
    return a * u * (u <= -lambda_)

# ===========================
# ROBUST LOGISTIC REGRESSION WITH SHIFT PARAMETER ESTIMATION
# ===========================
def train_robust_logistic_regression(X, y, lr=0.01, epochs=1000, tol=1e-6, lambda_=1.0, a=2.0, threshold_type='soft'):
    """
    Train robust logistic regression using gradient descent with shift parameter estimation.
    """
    m, n = X.shape
    theta = np.zeros(n)
    gamma = np.zeros(m)
    prev_loss = float('inf')

    for epoch in range(epochs):
        # Step 1: Update weights theta using logistic regression with offset
        z = np.dot(X, theta)
        u = y * (z - gamma)
        h = sigmoid(z - gamma)
        gradient_theta = np.dot(X.T, (h - (y + 1) / 2)) / m
        theta -= lr * gradient_theta

        # Step 2: Update shift parameters gamma using thresholding
        z = np.dot(X, theta)
        u = y * z
        if threshold_type == 'soft':
            gamma = soft_threshold(u, lambda_, a)
        elif threshold_type == 'hard':
            gamma = hard_threshold(u, lambda_, a)
        else:
            raise ValueError("threshold_type must be 'soft' or 'hard'")

        # Step 3: Compute current loss with L1 penalty
        loss = np.mean(np.log(1 + np.exp(-u + gamma))) + lambda_ * np.sum(np.abs(gamma))

        # Early stopping condition
        if abs(prev_loss - loss) < tol:
            break
        prev_loss = loss

    return theta, gamma

# ===========================
# PREDICTION FUNCTION
# ===========================
def predict_robust_logistic_regression(X, theta):
    """
    Make predictions using trained robust logistic regression model.
    """
    probabilities = sigmoid(np.dot(X, theta))
    return np.where(probabilities >= 0.5, 1, -1)

def print_average_with_se(data_list):
    """
    Calculates and prints the average of a list with an error bar of ±SE.

    Args:
        data_list (list): The list of numerical data.
    """

    if not data_list:
        print("Error: Input list is empty.")
        return

    try:
        mean = statistics.mean(data_list)
        stdev = statistics.stdev(data_list)
        se = stdev / math.sqrt(len(data_list))

        print(f"Average: {mean:.4f} ± {se:.4f} SE")

    except statistics.StatisticsError:
        print("Error: Cannot calculate standard deviation. List must contain at least two elements.")
    except TypeError:
        print("Error: List elements must be numerical.")

def cross_validate_parameters(X, y, a_candidates, lambda_candidates, threshold_type='soft', n_splits=5):
    """
    Perform cross-validation to select the best parameters a and lambda_.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    best_a = None
    best_lambda_ = None
    best_error = float('inf')

    for a in a_candidates:
        for lambda_ in lambda_candidates:
            print(a)
            print(lambda_)
            print('---------------------------------------')
            cv_errors = []
            for train_index, val_index in kf.split(X):
                X_train, X_val = X[train_index], X[val_index]
                y_train, y_val = y[train_index], y[val_index]

                # Convert labels to -1 and 1
                y_train_np = 2 * y_train - 1
                y_val_np = 2 * y_val - 1

                # Train robust logistic regression
                theta, gamma = train_robust_logistic_regression(
                    X_train, y_train_np, lr=0.1, epochs=2000, tol=1e-6, lambda_=lambda_, a=a, threshold_type=threshold_type
                )

                # Predict on validation set
                y_pred = predict_robust_logistic_regression(X_val, theta)

                # Compute misclassification rate
                misclassification_rate = 1 - accuracy_score(y_val_np, y_pred)
                cv_errors.append(misclassification_rate)

            # Average misclassification rate over the folds
            avg_error = np.mean(cv_errors)

            # Update best parameters if current combination is better
            if avg_error < best_error:
                best_error = avg_error
                best_a = a
                best_lambda_ = lambda_

    return best_a, best_lambda_, best_error

# # Define candidate values for a and lambda_
# a_candidates = [1, 2, 3, 4, 5, float('inf')]
# lambda_candidates = [0.01, 0.1, 1.0, 10.0]

# # Perform cross-validation to select the best a and lambda_
# best_a, best_lambda_, best_error = cross_validate_parameters(X, y, a_candidates, lambda_candidates, threshold_type='hard')

# print(f"Best a: {best_a}, Best lambda_: {best_lambda_}, Best CV error: {best_error}")

for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=i)

    # Introduce label noise only to the training set
    y_train_noisy = introduce_label_noise(y_train, noise_percentage=0.1)  # x% noise

    # Convert labels to -1 and 1 for training and testing sets
    y_train_np = 2 * y_train_noisy.values - 1  # Convert 0 → -1 and 1 → 1
    y_test_np = 2 * y_test.values - 1          # Convert 0 → -1 and 1 → 1

    # Train robust logistic regression on your dataset
    # Best a: 1, Best lambda_: 10, Best CV error: 0.07998088089505735
    theta, gamma = train_robust_logistic_regression(
        X_train, y_train_np, lr=0.1, epochs=2000, tol=1e-6, lambda_=10, a=1, threshold_type="hard"
    )

    # Predict on test set
    y_pred = predict_robust_logistic_regression(X_test, theta)

    # Compute misclassification rate
    misclassification_rate = 1 - accuracy_score(y_test_np, y_pred)
    misclassification_rate_l1_lasso.append(misclassification_rate)

print(misclassification_rate_l1_lasso)
print(f"Average misclassification rate over 50 runs L1 Lasso:")
print_average_with_se(misclassification_rate_l1_lasso)

[0.08110065170166547, 0.09558291093410576, 0.09196234612599563, 0.08761766835626361, 0.08472121650977549, 0.09992758870383778, 0.09775524981897177, 0.08761766835626361, 0.10644460535843592, 0.09051412020275162, 0.09847936278059377, 0.09196234612599563, 0.09413468501086164, 0.08182476466328747, 0.09051412020275162, 0.09196234612599563, 0.07748008689355534, 0.09123823316437363, 0.08254887762490948, 0.08327299058653148, 0.06806661839246919, 0.08761766835626361, 0.09051412020275162, 0.07965242577842147, 0.09703113685734976, 0.08327299058653148, 0.09051412020275162, 0.08979000724112962, 0.0861694424330196, 0.09703113685734976, 0.07096307023895732, 0.1028240405503259, 0.08689355539464161, 0.08906589427950762, 0.08254887762490948, 0.08472121650977549, 0.09413468501086164, 0.08037653874004347, 0.08472121650977549, 0.07603186097031134, 0.08037653874004347, 0.10209992758870379, 0.07965242577842147, 0.08761766835626361, 0.08689355539464161, 0.09920347574221577, 0.08906589427950762, 0.076031860970

---

Elastic Net

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import statistics
import math

misclassification_rate_elastic_net = []

# Define column names based on the UCI Machine Learning Repository description
column_names = [
    "word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d",
    "word_freq_our", "word_freq_over", "word_freq_remove", "word_freq_internet",
    "word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will",
    "word_freq_people", "word_freq_report", "word_freq_addresses", "word_freq_free",
    "word_freq_business", "word_freq_email", "word_freq_you", "word_freq_credit",
    "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money",
    "word_freq_hp", "word_freq_hpl", "word_freq_george", "word_freq_650",
    "word_freq_lab", "word_freq_labs", "word_freq_telnet", "word_freq_857",
    "word_freq_data", "word_freq_415", "word_freq_85", "word_freq_technology",
    "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct",
    "word_freq_cs", "word_freq_meeting", "word_freq_original", "word_freq_project",
    "word_freq_re", "word_freq_edu", "word_freq_table", "word_freq_conference",
    "char_freq_;", "char_freq_(", "char_freq_[", "char_freq_!", "char_freq_$",
    "char_freq_#", "capital_run_length_average", "capital_run_length_longest",
    "capital_run_length_total", "spam"
]

file_path = "C:/Users/91959/Desktop/CODE/Robust-Logistic-Regression-with-Shift-Parameter-Estimation/Robust Logistic Regression [DATA DIRECTORY]/Linear Case/spambase/spambase.data"

df = pd.read_csv(file_path, header=None, names=column_names)

# Separate features and target variable
X = df.drop('spam', axis=1)  # Features
y = df['spam']  # Target variable

# Standardize the feature set (zero mean, unit variance)
scaler = StandardScaler()
X = scaler.fit_transform(X)

def introduce_label_noise(y, noise_percentage=0.1):
    """
    Introduces label noise by flipping a percentage of majority class labels to the minority class.
    """
    value_counts = y.value_counts()
    majority_class = value_counts.idxmax()
    minority_class = value_counts.idxmin()

    majority_indices = y[y == majority_class].index
    num_noise = int(len(majority_indices) * noise_percentage)

    noise_indices = np.random.choice(majority_indices, num_noise, replace=False)

    y_noisy = y.copy()
    y_noisy.loc[noise_indices] = minority_class

    return y_noisy

# Methods for LogReg
# ===========================
# SIGMOID FUNCTION
# ===========================
def sigmoid(z):
    """Compute the sigmoid function."""
    return 1 / (1 + np.exp(-z))

# ===========================
# THRESHOLDING FUNCTIONS FOR SHIFT PARAMETERS
# ===========================
def soft_threshold(u, lambda_, a):
    """Soft thresholding function for shift parameter estimation."""
    return a * np.minimum(u + lambda_, 0)

def hard_threshold(u, lambda_, a):
    """Hard thresholding function for shift parameter estimation."""
    return a * u * (u <= -lambda_)

# ===========================
# ROBUST LOGISTIC REGRESSION WITH ELASTIC NET
# ===========================
def train_robust_logistic_regression_elastic(X, y, lr=0.01, epochs=1000, tol=1e-6, lambda_=1.0, alpha=0.5, a=2.0, threshold_type='soft'):
    """
    Train robust logistic regression using gradient descent with shift parameter estimation and Elastic Net regularization.
    """
    m, n = X.shape
    theta = np.zeros(n)
    gamma = np.zeros(m)
    prev_loss = float('inf')

    for epoch in range(epochs):
        # Step 1: Update weights theta using logistic regression with offset
        z = np.dot(X, theta)
        u = y * (z - gamma)
        h = sigmoid(z - gamma)
        gradient_theta = np.dot(X.T, (h - (y + 1) / 2)) / m

        # Elastic Net penalty gradient:
        l1_grad = alpha * np.sign(theta)  # L1 (Lasso)
        l2_grad = (1 - alpha) * theta  # L2 (Ridge)

        # Apply Elastic Net regularization
        gradient_theta += lambda_ * (l1_grad + l2_grad)

        # Update weights
        theta -= lr * gradient_theta

        # Step 2: Update shift parameters gamma using thresholding
        z = np.dot(X, theta)
        u = y * z
        if threshold_type == 'soft':
            gamma = soft_threshold(u, lambda_, a)
        elif threshold_type == 'hard':
            gamma = hard_threshold(u, lambda_, a)
        else:
            raise ValueError("threshold_type must be 'soft' or 'hard'")

        # Step 3: Compute current loss with Elastic Net penalty
        l1_term = alpha * np.sum(np.abs(theta))
        l2_term = (1 - alpha) * np.sum(theta**2)
        loss = np.mean(np.log(1 + np.exp(-u + gamma))) + lambda_ * (l1_term + l2_term)

        # Early stopping condition
        if abs(prev_loss - loss) < tol:
            break
        prev_loss = loss

    return theta, gamma

# ===========================
# PREDICTION FUNCTION
# ===========================
def predict_robust_logistic_regression(X, theta):
    """
    Make predictions using trained robust logistic regression model.
    """
    probabilities = sigmoid(np.dot(X, theta))
    return np.where(probabilities >= 0.5, 1, -1)

def print_average_with_se(data_list):
    """
    Calculates and prints the average of a list with an error bar of ±SE.
    """
    if not data_list:
        print("Error: Input list is empty.")
        return

    try:
        mean = statistics.mean(data_list)
        stdev = statistics.stdev(data_list)
        se = stdev / math.sqrt(len(data_list))

        print(f"Average: {mean:.4f} ± {se:.4f} SE")

    except statistics.StatisticsError:
        print("Error: Cannot calculate standard deviation. List must contain at least two elements.")
    except TypeError:
        print("Error: List elements must be numerical.")

def cross_validate_parameters(X, y, a_candidates, lambda_candidates, alpha_candidates, threshold_type='soft', n_splits=5):
    """
    Perform cross-validation to select the best parameters a, lambda_, and alpha.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    best_a = None
    best_lambda_ = None
    best_alpha = None
    best_error = float('inf')

    for a in a_candidates:
        for lambda_ in lambda_candidates:
            for alpha in alpha_candidates:
                print(a)
                print(lambda_)
                print(alpha)
                print('--------------------------')
                cv_errors = []
                for train_index, val_index in kf.split(X):
                    X_train, X_val = X[train_index], X[val_index]
                    y_train, y_val = y[train_index], y[val_index]

                    # Convert labels to -1 and 1
                    y_train_np = 2 * y_train - 1
                    y_val_np = 2 * y_val - 1

                    # Train robust logistic regression with Elastic Net
                    theta, gamma = train_robust_logistic_regression_elastic(
                        X_train, y_train_np, lr=0.1, epochs=2000, tol=1e-6, lambda_=lambda_, alpha=alpha, a=a, threshold_type=threshold_type
                    )

                    # Predict on validation set
                    y_pred = predict_robust_logistic_regression(X_val, theta)

                    # Compute misclassification rate
                    misclassification_rate = 1 - accuracy_score(y_val_np, y_pred)
                    cv_errors.append(misclassification_rate)

                # Average misclassification rate over the folds
                avg_error = np.mean(cv_errors)

                # Update best parameters if current combination is better
                if avg_error < best_error:
                    best_error = avg_error
                    best_a = a
                    best_lambda_ = lambda_
                    best_alpha = alpha

    return best_a, best_lambda_, best_alpha, best_error

# # Define candidate values for a, lambda_, and alpha
# a_candidates = [1, 2, 3, 4, 5, float('inf')]
# lambda_candidates = [0.01, 0.1, 1.0, 10.0]
# alpha_candidates = [0.1, 0.5, 0.9]  # Values for alpha (balance between L1 and L2)

# # Perform cross-validation to select the best a, lambda_, and alpha
# best_a, best_lambda_, best_alpha, best_error = cross_validate_parameters(X, y, a_candidates, lambda_candidates, alpha_candidates, threshold_type='hard')

# print(f"Best a: {best_a}, Best lambda_: {best_lambda_}, Best alpha: {best_alpha}, Best CV error: {best_error}")

# Now run the 50 iteration experiment with the best parameters
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=i)

    # Introduce label noise only to the training set
    y_train_noisy = introduce_label_noise(y_train, noise_percentage=0.1)  # x% noise

    # Convert labels to -1 and 1 for training and testing sets
    y_train_np = 2 * y_train_noisy.values - 1  # Convert 0 → -1 and 1 → 1
    y_test_np = 2 * y_test.values - 1          # Convert 0 → -1 and 1 → 1

    # Train robust logistic regression on your dataset with the best parameters
    # Best a: 1, Best lambda_: 0.01, Best alpha: 0.1, Best CV error: 0.08606642118680077
    theta, gamma = train_robust_logistic_regression_elastic(
        X_train, y_train_np, lr=0.1, epochs=2000, tol=1e-6, lambda_=0.01, alpha=0.1, a=1, threshold_type="hard"
    )

    # Predict on test set
    y_pred = predict_robust_logistic_regression(X_test, theta)

    # Compute misclassification rate
    misclassification_rate = 1 - accuracy_score(y_test_np, y_pred)
    misclassification_rate_elastic_net.append(misclassification_rate)

print(misclassification_rate_elastic_net)
print(f"Average misclassification rate over 50 runs Elastic Net:")
print_average_with_se(misclassification_rate_elastic_net)

[0.08182476466328747, 0.07965242577842147, 0.09123823316437363, 0.08906589427950762, 0.09051412020275162, 0.09413468501086164, 0.08761766835626361, 0.09268645908761763, 0.10065170166545978, 0.09558291093410576, 0.09196234612599563, 0.08979000724112962, 0.09485879797248375, 0.08037653874004347, 0.08472121650977549, 0.09341057204923964, 0.08254887762490948, 0.08689355539464161, 0.09485879797248375, 0.08689355539464161, 0.07458363504706733, 0.08182476466328747, 0.08110065170166547, 0.07820419985517746, 0.10065170166545978, 0.08979000724112962, 0.09558291093410576, 0.08399710354815348, 0.08472121650977549, 0.0861694424330196, 0.07241129616220132, 0.10789283128167992, 0.08399710354815348, 0.08906589427950762, 0.08399710354815348, 0.09196234612599563, 0.08979000724112962, 0.08327299058653148, 0.07892831281679946, 0.07530774800868933, 0.08182476466328747, 0.10137581462708178, 0.08689355539464161, 0.08327299058653148, 0.0861694424330196, 0.10861694424330193, 0.08979000724112962, 0.083272990586

---

In [3]:
data = {
    'Noise Level': ['10% Noise', '20% Noise'],
    'L1 Lasso': [
        '0.0879 ± 0.0011 SE',
        '0.0894 ± 0.0010 SE'
    ],
    'Elastic Net': [
        '0.0878 ± 0.0011 SE',
        '0.0881 ± 0.0010 SE'
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Set the 'Noise Level' column as the index
df.set_index('Noise Level', inplace=True)

# Write to Excel file
df.to_csv('Spambase_Results.csv')

print("Excel file has been created successfully.")

Excel file has been created successfully.
